In [7]:
import numpy as np
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [8]:
df = pd.read_csv("winemag-data-130k-v2.csv")

In [9]:
df = df[df['price'].notna()]
df = df[df['country'] == 'US']

In [10]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [22]:
X_t, Y = df["description"] + df["title"], df["price"]

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_t, Y, random_state=1)

In [40]:
docs_train = [nlp(d).vector for d in X_train]
X_train_w2v = np.vstack(docs_train)
X_train_w2v = pd.DataFrame(data=X_train_w2v)

(40698, 300)

In [47]:
ridge_w2v = Ridge().fit(X_train_w2v, y_train)

In [45]:
docs_test = [nlp(d).vector for d in X_test]
X_test_w2v = np.vstack(docs_test)
X_test_w2v = pd.DataFrame(data=X_test_w2v)

In [48]:
ridge_w2v.score(X_test_w2v, y_test)

0.32473371553038166

#### Combining Spacy and Bag Of Words

In [69]:
vect = CountVectorizer()
X_train_bow = vect.fit_transform(X_train)
X_test_bow = vect.transform(X_test)

In [70]:
X_train_bow = pd.DataFrame.sparse.from_spmatrix(X_train_bow)
X_test_bow = pd.DataFrame.sparse.from_spmatrix(X_test_bow)

In [71]:
X_train_combo = pd.concat([X_train_w2v, X_train_bow], axis=1)
X_test_combo = pd.concat([X_test_w2v, X_test_bow], axis=1)

In [ ]:
ridge_combo = Ridge().fit(X_train_combo, y_train)

In [ ]:
ridge_combo.score(X_test_combo, y_test)